## import

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
import re

from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
from brainspace.gradient import GradientMaps
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels

from scipy import io

In [ ]:
import os

import nibabel as nib
from nilearn.datasets import fetch_surf_fsaverage

import brainstat
from brainstat.stats.terms import FixedEffect

import pandas as pd

import scipy as sp
import scipy.stats
from scipy import stats

from math import pi
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

In [ ]:
import sys , numpy as np
mod = sys.modules[__name__]

## load cortex files

In [ ]:
from glob import glob
path = './data/schaefer200/cortex/'
filepaths = glob(path+'*.csv') # load path of csv files

In [ ]:
import pandas as pd
for i in range(len(filepaths)):
    dat = pd.read_csv(filepaths[i], header=None)
    res = re.split('[\\\/]',filepaths[i])
    globals()[res[-1][:-4]+'_a'] = dat.to_numpy() # load csv files as numpy type

In [ ]:
for i in range(1,100+1):
    globals()['ctx{}'.format(i)] = getattr(mod,'g{}_200_a'.format(i))

### make cortico-cortical connectivity matrix

In [ ]:
from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure 
correlation_measure = ConnectivityMeasure(kind='correlation')

for i in range(1,100+1):
    globals()['ctx{}_c'.format(i)] = correlation_measure.fit_transform([getattr(mod, 'ctx{}'.format(i))]) 

In [ ]:
for i in range(1, 100+1):
    globals()['ctx{}_c'.format(i)] = getattr(mod, 'ctx{}_c'.format(i))[0]
# (1,200,200) array shape -> (200,200) array shape

### 1. z-transformation
### 2. transform all diagonals into zeros

In [ ]:
tab = np.zeros(shape=(200,200))

In [ ]:
for i in range(1,101):
    tab = np.arctanh(getattr(mod, 'ctx{}_c'.format(i)))
    np.fill_diagonal(tab, np.zeros(200))
    globals()['ctx{}_cz'.format(i)] = tab

### get group connectivity matrix averaging z-transformed 100 connectivity matrix

In [ ]:
import numpy
ctx_sum = numpy.zeros(shape=(200,200))
for i in range(1,101):
    ctx_sum = ctx_sum + getattr(mod, 'ctx{}_cz'.format(i))

In [ ]:
ctx_mean = ctx_sum/100 

In [ ]:
plotting.plot_matrix(ctx_mean, vmin=0, vmax=1,colorbar=False) # we get group connectivity matrix of 100 individual connectivity matrices

## make gradient (henceforth eigenvector)

In [ ]:
labeling = load_parcellation('schaefer', scale=200, join=True)
mask = labeling != 0
# generate labeing and mask based on schaefer200

In [ ]:
surf_lh, surf_rh = load_conte69()

### make eigenvectors of group connectivity matrix

In [ ]:
gradients_kernel = [None] * 3 

gm = GradientMaps(kernel='normalized_angle', approach='dm', random_state=0)
gm.fit(ctx_mean) 
    
for i in range(0,2+1):
    gradients_kernel[i] = map_to_labels(gm.gradients_[:,i], labeling, mask=mask,
                                       fill=np.nan)

label_text = ['eigenvector1','eigenvector2','eigenvector3']
plot_hemispheres(surf_lh, surf_rh, array_name=gradients_kernel, size=(1200,600),
                cmap='viridis', color_bar=True, label_text=label_text, zoom=1.45, color_range = (-0.1,0.1)) 

### make template of eigenvectors 1,2 and 3 with hcp

In [ ]:
mat_file_name = "./data/gradients_FC.mat"
mat_file = scipy.io.loadmat(mat_file_name)

In [ ]:
gm_mean = mat_file['gm_mean']

In [ ]:
gradients_kernel = [None] * 3 

gm = GradientMaps(kernel='normalized_angle', approach='dm', random_state=0) 

for i in range(0,2+1):
    gradients_kernel[i] = map_to_labels(gm_mean[:,i], labeling, mask=mask,
                                       fill=np.nan)

label_text = ['eigenvector1','eigenvector2','eigenvector3']
plot_hemispheres(surf_lh, surf_rh, array_name=gradients_kernel, size=(1200,600),
                cmap='viridis', color_bar=True, label_text=label_text, zoom=1.45, color_range = (-0.1,0.1)) 

### align group connectivity matrix with hcp eigenvectors

In [ ]:
# eigenvector alignment
align = GradientMaps(kernel='normalized_angle', approach='dm', alignment='procrustes', n_components = 5)

align.fit(ctx_mean, reference = gm_mean)

In [ ]:
gradients_kernel = [None] * 3 
for i in range(0,2+1):
    gradients_kernel[i] = map_to_labels(align.aligned_[:, i], labeling, mask=mask,
                                       fill=np.nan)

    
label_text = ['Eigenvector1','Eigenvector2','Eigenvector3']
plot_hemispheres(surf_lh, surf_rh, array_name=gradients_kernel, size=(1200,600),
                cmap='viridis', color_bar=False, label_text=label_text, zoom=1.45, color_range = (-0.1, 0.1)) 

### align each individual connectivity matrix with hcp-aligned group connectivity matrix

> we call it individual eigenvector

In [ ]:
for i in range(1,100+1):
    align2 = GradientMaps(kernel='normalized_angle', approach='dm', alignment='procrustes', n_components = 5)
    align2.fit(getattr(mod,'ctx{}_cz'.format(i)), reference = align.aligned_)
    globals()['grad_{}'.format(i)] = align2.aligned_[:,0:2+1]

## Multivariate analysis

> we perform multivariate analysis with 100 individual eigenvectors

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

brain = pd.read_excel("./data/환자군_정상군_181104_결측값업데이트_scores2_bypark.xlsx")

In [ ]:
brain

make sex and grp as dummy variables

In [ ]:
age = brain['Age']
sex = brain['Sex']
sex=np.array(sex)
sex=sex.reshape(100,1)
grp = brain['group']
grp=np.array(grp)
grp=grp.reshape(100,1)
NumSubj = np.size(grp)

In [ ]:
sex_dum1 = np.zeros((NumSubj,1))
sex_dum1 = np.where(sex==0, 1, sex_dum1)
sex_dum2 = np.zeros((NumSubj,1))
sex_dum2 = np.where(sex==1, 1, sex_dum2)
sex_dum = np.concatenate((sex_dum1, sex_dum2), axis=1) # F,M

In [ ]:
grp_dum1 = np.zeros((NumSubj,1))
grp_dum1 = np.where(grp==0, 1, grp_dum1)
grp_dum2 = np.zeros((NumSubj, 1))
grp_dum2 = np.where(grp==1, 1, grp_dum2)
grp_dum = np.concatenate((grp_dum1, grp_dum2), axis=1) # ctl pat

In [ ]:
# get brainstat dir
brainstat_dir = os.path.dirname(brainstat.__file__)

define model

In [ ]:
# from brainstat.stats.terms import FixedEffect

term_age = FixedEffect(age, "age")
term_sex = FixedEffect(sex_dum, {"F", "M"})
term_grp = FixedEffect(grp_dum, {"ctl", "pat"})
model = term_age + term_sex + term_grp

model

append 200x3 matrices of 100 individuals by 3d array

In [ ]:
Z = np.dstack([grad_1,grad_2])
for i in range(3,100+1):
    Z = np.dstack([Z, getattr(mod, 'grad_{}'.format(i))])

In [ ]:
NumROI = 200

In [ ]:
data = np.zeros((NumSubj, NumROI, 3))
for ns in range(0,NumSubj):
    mat = Z[:,:,ns]
    data[ns,:,:] = mat

In [ ]:
from brainstat.stats.SLM import SLM

contrast = grp_dum[:,0] - grp_dum[:,1]
slm = SLM(model, contrast, correction="fdr")
slm.fit(data)

when we put contrast = patient groups - normal controls, we get t-values of these contrasts

In [ ]:
slm.t

we save t-values of group differences in cortical eigenvectors for future correlation analysis with neurotransmitter distribution

In [ ]:
pd.DataFrame([slm.t[0]]).T.to_csv("./fig1/total_t_value.csv", header=False, index=False)

we will get p-values

In [ ]:
rv = sp.stats.t(df=slm.df)
cdf = rv.cdf(slm.t)
pval = 1-cdf

In [ ]:
pval

we will get corrected p-values

In [ ]:
# two-tailed
pval = stats.t.sf(np.abs(slm.t), slm.df)*2

from  statsmodels.stats import multitest
pcor = multitest.fdrcorrection(pval.ravel())

In [ ]:
pcor[1]

return significant indexes of t-values under corrected p-values < 0.055

In [ ]:
sig_idx = np.where(pcor[1]<0.055)

In [ ]:
tot_t = slm.t.T
sig_t = np.zeros((200,1))
sig_t[sig_idx] = tot_t[sig_idx]

In [ ]:
sig_t

import maximum value among significant values

In [ ]:
np.max(sig_t)

we only plot significant regions onto brain surface 

In [ ]:
data = [None]*1
data[0] = map_to_labels(sig_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['sig_t']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,np.max(sig_t)), zoom=1)

## visualization

### radar chart

classify 200 nodes into 7 networks

In [ ]:
Vis_idx = list(range(1-1,14+1-1)) + list(range(101-1,115+1-1))
SomMot_idx = list(range(15-1,30+1-1)) + list(range(116-1,134+1-1))
DorsAttn_idx = list(range(31-1,43+1-1)) + list(range(135-1,147+1-1))
SalVentAttn_idx = list(range(44-1,54+1-1)) + list(range(148-1,158+1-1))
Limbic_idx = list(range(55-1,60+1-1)) + list(range(159-1,164+1-1))
Cont_idx = list(range(61-1,73+1-1)) + list(range(165-1,181+1-1))
Default_idx = list(range(74-1,100+1-1)) + list(range(182-1,200+1-1))

In [ ]:
Vis_m = np.mean(tot_t[Vis_idx])
SomMot_m = np.mean(tot_t[SomMot_idx])
DorsAttn_m = np.mean(tot_t[DorsAttn_idx])
SalVentAttn_m = np.mean(tot_t[SalVentAttn_idx])
Limbic_m = np.mean(tot_t[Limbic_idx])
Cont_m = np.mean(tot_t[Cont_idx])
Default_m = np.mean(tot_t[Default_idx])

In [ ]:
df = pd.DataFrame({
    'Visual': [Vis_m],
    'Somatomotor': [SomMot_m],
    'Dorsal attention': [DorsAttn_m],
    'Ventral attention': [SalVentAttn_m],
    'Limbic': [Limbic_m],
    'Frontoparietal': [Cont_m],
    'Default mode': [Default_m]
})

In [ ]:
y = df.values[0]
y = numpy.concatenate((y,y[0]),axis=None)

make spider-plot

In [ ]:
labels = df.columns[:]
num_labels = len(labels)

labels = ['','','','','','','']

angles = [x/float(num_labels)*(2*pi) for x in range(num_labels)] 
angles += angles[:1] 


fig = plt.figure(figsize=(5,5))
# fig.set_facecolor('white')
 
    
ax = plt.subplot(polar=True)
ax.set_theta_offset(pi / 2) ## starting point
ax.set_theta_direction(-1) ## direction
    
plt.xticks(angles[:-1], labels, fontsize=13) ## x-ticks label
ax.tick_params(axis='x', which='major', pad=15) ## make empty spaces between x-axes and ticks
 
ax.set_rlabel_position(0) ## y-axis degree
plt.yticks([1,1.5,2,2.5],['1','1.5','2','2.5'], fontsize=20) ## y-ticks
plt.ylim(1,2.5)
    
ax.plot(angles, y, linewidth=2, linestyle='solid', color='black') ## print spider-plot     

return values of spider plot

In [ ]:
df

we make network-wide brain surfaces to append spider-plot labels.

In [ ]:
Vis_t = np.zeros((200,1))
Vis_t[Vis_idx] = tot_t[Vis_idx]

SomMot_t = np.zeros((200,1))
SomMot_t[SomMot_idx] = tot_t[SomMot_idx]

DorsAttn_t = np.zeros((200,1))
DorsAttn_t[DorsAttn_idx] = tot_t[DorsAttn_idx]

SalVentAttn_t = np.zeros((200,1))
SalVentAttn_t[SalVentAttn_idx] = tot_t[SalVentAttn_idx]

Limbic_t = np.zeros((200,1))
Limbic_t[Limbic_idx] = tot_t[Limbic_idx]

Cont_t = np.zeros((200,1))
Cont_t[Cont_idx] = tot_t[Cont_idx]

Default_t = np.zeros((200,1))
Default_t[Default_idx] = tot_t[Default_idx]

In [ ]:
data = [None]*1
data[0] = map_to_labels(Vis_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['Vis_t']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,np.max(Vis_t)), zoom=1)

In [ ]:
data = [None]*1
data[0] = map_to_labels(SomMot_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['SomMot_t']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,np.max(sig_t)), zoom=1)

In [ ]:
data = [None]*1
data[0] = map_to_labels(DorsAttn_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['DorsAttn_t']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,np.max(sig_t)), zoom=1)

In [ ]:
data = [None]*1
data[0] = map_to_labels(SalVentAttn_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['SalVentAttn_t']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,np.max(sig_t)), zoom=1)

In [ ]:
data = [None]*1
data[0] = map_to_labels(Limbic_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['Limbic_t']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,np.max(sig_t)), zoom=1)

In [ ]:
data = [None]*1
data[0] = map_to_labels(Cont_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['Cont_t']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,np.max(sig_t)), zoom=1)

In [ ]:
data = [None]*1
data[0] = map_to_labels(Default_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['Default_t']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,np.max(sig_t)), zoom=1)

not only significant regions, but also all regions

In [ ]:
data = [None]*1
data[0] = map_to_labels(tot_t[:,0], labeling, mask=mask, fill=np.nan)


label_text = ['whole t_value']
plot_hemispheres(surf_lh, surf_rh, array_name=data, size=(1200,600),
                cmap='Reds', color_bar=False, label_text=label_text, color_range=(0,4), zoom=1)

### scree plot

we make scree plot of eigenvectors

In [ ]:
ctx_df = pd.DataFrame(ctx_mean)

In [ ]:
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
scaled_sum = ctx_mean.copy()

In [ ]:
scaled_sum = pd.DataFrame(scaler.fit_transform(scaled_sum))                         

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=5)

In [ ]:
Principal_components = pca.fit_transform(scaled_sum)

In [ ]:
pca_df = pd.DataFrame(data = Principal_components, columns = ['PC1', 'PC2', 'PC3', 'PC4','PC5'])

In [ ]:
print(pca_df)

In [ ]:
pca_fit = pca.fit(scaled_sum)

In [ ]:
PC_values = np.arange(pca.n_components_) + 1

In [ ]:
# Scree plot

fig = plt.figure(figsize=(7,5))
fig.set_facecolor('white')

plt.plot(PC_values, pca.explained_variance_ratio_, 'ro-', linewidth=2, color='black')
plt.xticks([1,2,3,4,5])
plt.xlabel('Component number')
plt.ylabel('Scaled eigenvalues')
plt.show()
# plt.axis([0,5,0,3])

we check explained portion

In [ ]:
pca.explained_variance_ratio_

In [ ]:
print("Proportion of Variance Explained : ",
     pca.explained_variance_ratio_.round(2))

out_sum = np.cumsum(pca.explained_variance_ratio_).round(2)
print("Cumulative Prop. Variance Explained", out_sum)

### yeo17 atlas for functional connectivity schema

Load atlases

In [ ]:
from nilearn.maskers import NiftiMasker
from nilearn.input_data import NiftiMasker

In [ ]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

In [ ]:
from nilearn import datasets
yeo = datasets.fetch_atlas_yeo_2011()

In [ ]:
data = datasets.fetch_development_fmri(n_subjects=50)

extract coordinates on atlas - parcellation

In [ ]:
connectome_measure = ConnectivityMeasure(kind='correlation')

from nilearn import plotting

masker = NiftiLabelsMasker(labels_img=yeo['thick_17'],standardize=True,memory='nilearn_cache')

In [ ]:
time_series = []
for func, confounds in zip(data.func, data.confounds):
    time_series.append(masker.fit_transform(func, confounds=confounds))

In [ ]:
correlation_matrices = connectome_measure.fit_transform(time_series)

In [ ]:
mean_correlation_matrix = connectome_measure.mean_

In [ ]:
coordinates_l = plotting.find_parcellation_cut_coords(labels_img=yeo['thick_17'], label_hemisphere='left')

In [ ]:
coordinates_r = plotting.find_parcellation_cut_coords(labels_img=yeo['thick_17'], label_hemisphere='right')

In [ ]:
plotting.plot_connectome(mean_correlation_matrix, coordinates_l,
                        edge_threshold="80%", annotate=False)
# plt.savefig('./fig1/yeo17_left.png')

In [ ]:
plotting.plot_connectome(mean_correlation_matrix, coordinates_r,
                        edge_threshold="80%", annotate=False)
# plt.savefig('./fig1/yeo17_right.png')